In [1]:
import pandas as pd
import synthia as syn
import matplotlib.pyplot as plt

In [2]:
df_raw = pd.read_csv("../data/raw/predictive_maintenance.csv")

In [19]:
ds_true = syn.util.load_dataset(name='SAF-Synthetic')


In [20]:
df_raw["Type"].value_counts()

L    6000
M    2997
H    1003
Name: Type, dtype: int64

In [21]:
df_raw

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure
...,...,...,...,...,...,...,...,...,...,...
9995,9996,M24855,M,298.8,308.4,1604,29.5,14,0,No Failure
9996,9997,H39410,H,298.9,308.4,1632,31.8,17,0,No Failure
9997,9998,M24857,M,299.0,308.6,1645,33.4,22,0,No Failure
9998,9999,H39412,H,299.0,308.7,1408,48.5,25,0,No Failure


In [22]:
generator = syn.CopulaDataGenerator()
# We fit and store the original samples and use them to recontuct new synthetic samples.
generator.fit(df_raw["Process temperature [K]"].reset_index(), copula=syn.IndependenceCopula())
N_SAMPLES = 1000
ds_synth_empirical = generator.generate(N_SAMPLES)
ds_synth_empirical['temperature_fl'].plot.hist()
plt.xlabel('Air temperature in K')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
ds_synth_empirical

array([[8142.57780768,  309.6       ],
       [6144.76699123,  312.7       ],
       [3408.83312566,  311.1       ],
       ...,
       [7762.65934253,  313.        ],
       [1970.32527098,  310.1       ],
       [3614.65681954,  307.9       ]])